In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
torch.cuda.is_available()

False

In [3]:
tok = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased', use_fast=False)
model = AutoModel.from_pretrained('KB/bert-base-swedish-cased')


Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
#inputs = tok(['hälsa hälsan hälsans'], return_tensors="pt", padding='max_length', max_length = 512, truncation=True)
inputs = tok(X[:10].values.tolist(), return_tensors="pt", padding='max_length', max_length = 512, truncation=True)

In [29]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [30]:
inputs.input_ids[:10]

tensor([[    2,   301, 23754,  ...,    62,  2356,     3],
        [    2, 22086, 19935,  ..., 39988,  1964,     3],
        [    2, 24779, 49796,  ...,     0,     0,     0],
        ...,
        [    2, 23183,   399,  ...,     0,     0,     0],
        [    2,  1528,    84,  ...,     0,     0,     0],
        [    2,  3740,  2582,  ...,     0,     0,     0]])

In [33]:
inputs2 = {}
for key in inputs.keys():
    inputs2[key] = inputs[key][:10]

In [34]:
inputs2

{'input_ids': tensor([[    2,   301, 23754,  ...,    62,  2356,     3],
         [    2, 22086, 19935,  ..., 39988,  1964,     3],
         [    2, 24779, 49796,  ...,     0,     0,     0],
         ...,
         [    2, 23183,   399,  ...,     0,     0,     0],
         [    2,  1528,    84,  ...,     0,     0,     0],
         [    2,  3740,  2582,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [32]:
inputs2 = {'input_ids':inputs['input_ids'][:10], 'token_type_ids':inputs['token_type_ids'][:10], 'attention_mask':inputs['attention_mask'][:10] }

In [25]:
outputs = model(**inputs)
#outputs = outputs['pooler_output'].detach().numpy().reshape(X.shape[0], -1)

In [19]:
outputs['pooler_output'].detach().numpy().reshape(100,-1).shape

(100, 768)

In [107]:
class NLPTransformer(TransformerMixin):
    def __init__(self):
        print('Init called')
        self.model_name = 'KB/bert-base-swedish-cased'
        self.Bert = AutoModel.from_pretrained(self.model_name)
        self.Tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
    def fit(self, X, y=None):
        print('Fit called')
        
        
        return self
    
    def partial_fit(self, X, y=None):
        print('Partial Fit called')

        return self
        
    def transform(self, X):
        print('Transform Called')
       
        # Transform input tokens 
        transformed = []
        for text in X:
            #print(f'text: {text}')
            
            inputs = self.Tokenizer(text, return_tensors="pt", padding='max_length', max_length = 512, truncation=True)

            # Run Bert model
            outputs = self.Bert(**inputs)
            #outputs['last_hidden_state'].detach().numpy().reshape(-1,)
            outputs = outputs['pooler_output'].detach().numpy().reshape(-1,)
            print(f'output shape: {outputs.shape}')
            transformed.append(outputs)
        
        transformed = np.array(transformed)
        print(f'transformed.shape: {transformed.shape}')
        
        return transformed

In [46]:
class NLPTransformer(TransformerMixin):
    def __init__(self):
        print('Init called')
        self.model_name = 'KB/bert-base-swedish-cased'
        self.Bert = AutoModel.from_pretrained(self.model_name)
        self.Tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.batch_size = 10
        
    def fit(self, X, y=None):
        print('Fit called')
        
        
        return self
    
    def partial_fit(self, X, y=None):
        print('Partial Fit called')

        return self
        
    def transform(self, X):
        print('Transform Called')
       
        # Transform input tokens. This is most efficient if done in one batch 
        inputs = self.Tokenizer(X.values.tolist(), return_tensors="pt", padding='max_length', max_length = 512, truncation=True)

        # Run Bert model, We must mini batch this in order to not overflow the memory of the system
        transformed = []
        
        batches = int(np.ceil(X.shape[0] / self.batch_size))
        for batchId in range(batches):
            print(f'Running batch {batchId+1}/{batches}')
        
            inputs_batch = {}
            for key in inputs.keys():
                inputs_batch[key] = inputs[key][batchId * self.batch_size:(batchId + 1) * self.batch_size]

            outputs = self.Bert(**inputs_batch)
            outputs = outputs['pooler_output'].detach().numpy()
            print(f'output shape: {outputs.shape}')
            transformed.extend(outputs)
        
        transformed = np.array(transformed)
        print(f'transformed.shape: {transformed.shape}')
        
        return transformed

In [37]:
np.ceil(3.2)

4.0

In [47]:
pipeline = Pipeline([    
            ('nlpTransformer', NLPTransformer()),
            ('clf', RandomForestClassifier())])

Init called


Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [108]:
#pipeline = Pipeline([    
#            ('nlpTransformer', NLPTransformer()),
#            ('clf', SGDClassifier())])

Init called


Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df = pd.read_csv('../data/bbc-text.csv')

In [6]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [7]:
df['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [8]:
df2 = pd.get_dummies(df, columns=['category'])

In [9]:
X = df2['text']

In [10]:
Y = df2.drop(columns=['text'])

In [11]:
X.head()

0    tv future in the hands of viewers with home th...
1    worldcom boss  left books alone  former worldc...
2    tigers wary of farrell  gamble  leicester say ...
3    yeading face newcastle in fa cup premiership s...
4    ocean s twelve raids box office ocean s twelve...
Name: text, dtype: object

In [12]:
Y.head()

,category_business,category_entertainment,category_politics,category_sport,category_tech
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,1,0,0,0


In [13]:
#Test train split

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [66]:
X_train.shape

(1780,)

In [69]:
Y_train[:5]

,category_business,category_entertainment,category_politics,category_sport,category_tech
1490,0,1,0,0,0
2001,1,0,0,0,0
1572,1,0,0,0,0
1840,0,1,0,0,0
610,0,0,0,0,1


In [99]:
class MiniBatcher():
    def __init__(self, X, Y, batch_size=100, max_epochs=1):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.max_epochs = max_epochs
        self.batchId = 0
        self.epochId = 0
        
    def getBatchIterator(self):
        self.batchId = 0
        self.epochId = 0
        
        while True:
            X_mini = X[self.batchId * self.batch_size:(self.batchId + 1) * self.batch_size]
            Y_mini = Y[self.batchId * self.batch_size:(self.batchId + 1) * self.batch_size]
            self.batchId += 1
            print(self.batchId, self.epochId)
            print(len(X_mini))
            if len(X_mini) < self.batch_size:
                self.epochId += 1
                self.batchId = 0
                if self.epochId <= self.max_epochs:
                    print('will break')
                    break
            
            yield X_mini, Y_mini
            


In [100]:
mMiniBatcher = MiniBatcher(X_train, Y_train)

In [101]:
batchIterator = mMiniBatcher.getBatchIterator()

In [106]:
i = 0
for X_mini, Y_mini in batchIterator:
    print(f'Running batch {i}')
    output = pipeline.partial_fit(X_mini, Y_mini)
    i+=1
print('Training completed...')

2 0
100
Running batch 0


AttributeError: 'Pipeline' object has no attribute 'partial_fit'

In [61]:
output = pipeline.fit(X_train[:25], Y_train[:25].values)

Fit called
Transform Called
Running batch 1/3
output shape: (10, 768)
Running batch 2/3
output shape: (10, 768)
Running batch 3/3
output shape: (5, 768)
transformed.shape: (25, 768)


In [72]:
output.predict(X_test[:3])

Transform Called
Running batch 1/1
output shape: (3, 768)
transformed.shape: (3, 768)


array([[0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]], dtype=uint8)

In [73]:
probs = output.predict_proba(X_test[:3])

Transform Called
Running batch 1/1
output shape: (3, 768)
transformed.shape: (3, 768)


In [69]:
probs

[array([[0.95, 0.05],
        [0.1 , 0.9 ],
        [0.26, 0.74]]),
 array([[0.22, 0.78],
        [0.98, 0.02],
        [0.92, 0.08]]),
 array([[1.  , 0.  ],
        [1.  , 0.  ],
        [0.95, 0.05]]),
 array([[0.93, 0.07],
        [0.99, 0.01],
        [0.98, 0.02]]),
 array([[0.9 , 0.1 ],
        [0.93, 0.07],
        [0.89, 0.11]])]

In [74]:
Y_test[:3].values

array([[0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]], dtype=uint8)

In [55]:
Y_test[:10]

,category_business,category_entertainment,category_politics,category_sport,category_tech
414,0,0,1,0,0
420,1,0,0,0,0
1644,0,1,0,0,0
416,0,0,0,0,1
1232,0,0,0,1,0
1544,1,0,0,0,0
1748,1,0,0,0,0
1264,0,0,0,1,0
629,0,0,0,1,0
1043,0,0,0,0,1


In [ ]:
#Evaluate the model and print the accuracy

def EvaluateModel(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    
    total_hits = np.sum(np.sum(Y_pred == Y_test))
    total_misses = np.sum(np.sum(Y_pred != Y_test))
    total_accuracy = total_hits/(total_hits + total_misses)
       
    target_names = [name for name in Y.columns]

    precisions = []
    recalls = []
    f1scores = []
    for (name, col) in zip(target_names, range(len(target_names))):
        y_test = Y_test[name].values
        y_pred = Y_pred[:, col]
        
        if(np.max(y_test) <= 1):
            #Only one category
            precisions.append(precision_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred))
            f1scores.append(f1_score(y_test, y_pred))
        print(f'Category: {name}')
        print(classification_report(y_test, y_pred))
        print('-'*42)
     
    
    print(f'Total Accuracy: {total_accuracy}')
    print(f'Average Precission: {np.average(precisions)}')
    print(f'Average Recall: {np.average(recalls)}')
    print(f'Average F1 Score: {np.average(f1scores)}')

In [ ]:
EvaluateModel(pipeline, X_test[:20], Y_test[:20])

In [136]:
pipeline.predict_proba(XX[0:10])

Transform Called
text: tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also 

output shape: (768,)
text: ocean s twelve raids box office ocean s twelve  the crime caper sequel starring george clooney  brad pitt and julia roberts  has gone straight to number one in the us box office chart.  it took $40.8m (£21m) in weekend ticket sales  according to studio estimates. the sequel follows the master criminals as they try to pull off three major heists across europe. it knocked last week s number one  national treasure  into third place. wesley snipes  blade: trinity was in second  taking $16.1m (£8.4m). rounding out the top five was animated fable the polar express  starring tom hanks  and festive comedy christmas with the kranks.  ocean s twelve box office triumph marks the fourth-biggest opening for a december release in the us  after the three films in the lord of the rings trilogy. the sequel narrowly beat its 2001 predecessor  ocean s eleven which took $38.1m (£19.8m) on its opening weekend and $184m (£95.8m) in total. a remake of the 1960s film  starring frank

output shape: (768,)
text: last star wars  not for children  the sixth and final star wars movie may not be suitable for young children  film-maker george lucas has said.  he told us tv show 60 minutes that revenge of the sith would be the darkest and most violent of the series.  i don t think i would take a five or six-year-old to this   he told the cbs programme  to be aired on sunday. lucas predicted the film would get a us rating advising parents some scenes may be unsuitable for under-13s. it opens in the uk and us on 19 may. he said he expected the film would be classified pg-13 - roughly equivalent to a british 12a rating.  the five previous star wars films have all carried less restrictive pg - parental guidance - ratings in the us. in the uk  they have all been passed u - suitable for all - with the exception of attack of the clones  which got a pg rating in 2002. revenge of the sith - the third prequel to the original 1977 star wars film - chronicles the transformation of the

[array([[0.94, 0.06],
        [0.35, 0.65],
        [0.95, 0.05],
        [0.86, 0.14],
        [0.92, 0.08],
        [0.89, 0.11],
        [0.92, 0.08],
        [0.89, 0.11],
        [0.89, 0.11],
        [0.89, 0.11]]),
 array([[0.91, 0.09],
        [0.91, 0.09],
        [0.89, 0.11],
        [0.92, 0.08],
        [0.36, 0.64],
        [0.8 , 0.2 ],
        [0.78, 0.22],
        [0.73, 0.27],
        [0.76, 0.24],
        [0.61, 0.39]]),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 array([[0.87, 0.13],
        [0.83, 0.17],
        [0.27, 0.73],
        [0.25, 0.75],
        [0.77, 0.23],
        [0.65, 0.35],
        [0.61, 0.39],
        [0.52, 0.48],
        [0.46, 0.54],
        [0.66, 0.34]]),
 array([[0.28, 0.72],
        [0.91, 0.09],
        [0.89, 0.11],
        [0.97, 0.03],
        [0.95, 0.05],
        [0.66, 0.34],
        [0.69, 0.31],
        [0.86, 0.14],
        [0.89, 0

In [135]:
pipeline.predict_proba(XX[0:10])

Transform Called
text: tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also 

output shape: (768,)
text: ocean s twelve raids box office ocean s twelve  the crime caper sequel starring george clooney  brad pitt and julia roberts  has gone straight to number one in the us box office chart.  it took $40.8m (£21m) in weekend ticket sales  according to studio estimates. the sequel follows the master criminals as they try to pull off three major heists across europe. it knocked last week s number one  national treasure  into third place. wesley snipes  blade: trinity was in second  taking $16.1m (£8.4m). rounding out the top five was animated fable the polar express  starring tom hanks  and festive comedy christmas with the kranks.  ocean s twelve box office triumph marks the fourth-biggest opening for a december release in the us  after the three films in the lord of the rings trilogy. the sequel narrowly beat its 2001 predecessor  ocean s eleven which took $38.1m (£19.8m) on its opening weekend and $184m (£95.8m) in total. a remake of the 1960s film  starring frank

output shape: (768,)
text: last star wars  not for children  the sixth and final star wars movie may not be suitable for young children  film-maker george lucas has said.  he told us tv show 60 minutes that revenge of the sith would be the darkest and most violent of the series.  i don t think i would take a five or six-year-old to this   he told the cbs programme  to be aired on sunday. lucas predicted the film would get a us rating advising parents some scenes may be unsuitable for under-13s. it opens in the uk and us on 19 may. he said he expected the film would be classified pg-13 - roughly equivalent to a british 12a rating.  the five previous star wars films have all carried less restrictive pg - parental guidance - ratings in the us. in the uk  they have all been passed u - suitable for all - with the exception of attack of the clones  which got a pg rating in 2002. revenge of the sith - the third prequel to the original 1977 star wars film - chronicles the transformation of the

[array([[0.94, 0.06],
        [0.35, 0.65],
        [0.95, 0.05],
        [0.86, 0.14],
        [0.92, 0.08],
        [0.89, 0.11],
        [0.92, 0.08],
        [0.89, 0.11],
        [0.89, 0.11],
        [0.89, 0.11]]),
 array([[0.91, 0.09],
        [0.91, 0.09],
        [0.89, 0.11],
        [0.92, 0.08],
        [0.36, 0.64],
        [0.8 , 0.2 ],
        [0.78, 0.22],
        [0.73, 0.27],
        [0.76, 0.24],
        [0.61, 0.39]]),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 array([[0.87, 0.13],
        [0.83, 0.17],
        [0.27, 0.73],
        [0.25, 0.75],
        [0.77, 0.23],
        [0.65, 0.35],
        [0.61, 0.39],
        [0.52, 0.48],
        [0.46, 0.54],
        [0.66, 0.34]]),
 array([[0.28, 0.72],
        [0.91, 0.09],
        [0.89, 0.11],
        [0.97, 0.03],
        [0.95, 0.05],
        [0.66, 0.34],
        [0.69, 0.31],
        [0.86, 0.14],
        [0.89, 0

In [130]:
y.iloc[0:10].values

array([[0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]], dtype=uint8)

In [100]:
XX[0:2]

['tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to hig